In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import requests

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rnrib\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rnrib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
reviews = pd.read_csv("reviews.csv")

,Id,title,summary,text
0,1882931173,Its Only Art If Its Well Hung!,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


Word Count Distribution in Reviews

In [ ]:
reviews.word_count.value_counts().sort_index().plot(kind='line',
                                                    title='Word Count Distribution in Reviews',
                                                    legend=False, figsize=(10, 6),
                                                    xlabel='Word Count',
                                                    ylabel='Frequency')

Word Count Interval Distribution in Reviews

In [ ]:
max_word_count = math.ceil(reviews['word_count'].max() / 1000) * 1000
word_count_intervals = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, max_word_count]
word_count_distribution = pd.cut(reviews['word_count'], bins=word_count_intervals).value_counts().sort_index()
plt.figure(figsize=(8, 6))
bars = plt.bar(word_count_distribution.index.astype(str), word_count_distribution.values)
plt.title('Word Count Interval Distribution in Reviews')
plt.xlabel('Word Count Intervals')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height), xytext=(0, 3),
                textcoords="offset points", ha='center', va='bottom')

plt.tight_layout()
plt.show()

Word Count Interval Distribution in Reviews (Percentage)

In [ ]:
word_count_distribution = pd.cut(reviews['word_count'], bins=word_count_intervals).value_counts(normalize=True).sort_index()

# Create a bar chart to visualize the interval distribution in percentage
plt.figure(figsize=(8, 6))
bars = plt.bar(word_count_distribution.index.astype(str), word_count_distribution.values)
plt.title('Word Count Interval Distribution in Reviews (Percentage)')
plt.xlabel('Word Count Intervals')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=45)
# Annotate each bar with its exact percentage value
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{100*height:2.2f}%', xy=(bar.get_x() + bar.get_width() / 2, height), xytext=(0, 3),
                textcoords="offset points", ha='center', va='bottom')

plt.tight_layout()
plt.show()


Average Word Length

In [ ]:
reviews['average_word_length'].plot(kind='kde')
plt.xlim(0, 20)

Average Word Length Distribution in Reviews

In [ ]:
reviews.average_word_length.value_counts().sort_index().plot(kind='line', title='Average Word Length Distribution in Reviews', legend=False, figsize=(10, 6), rot=0, y='text', xlabel='Average Word Length', ylabel='Frequency', xlim=(0, 100))

In [ ]:
reviews.rounded_average_word_length.value_counts().sort_index().plot(kind='bar', title='Rounded Average Word Length Distribution in Reviews', legend=False, figsize=(10, 6), rot=0, y='text', xlabel='Rounded Average Word Length', ylabel='Frequency', xlim=(0, 50)).bar_label(container=plt.gca().containers[0])

In [ ]:
average_word_length_intervals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, float('inf')]
average_word_length_distribution = pd.cut(reviews['average_word_length'], bins=average_word_length_intervals).value_counts().sort_index()

plt.figure(figsize=(8, 6))
bars = plt.bar(average_word_length_distribution.index.astype(str), average_word_length_distribution.values)
plt.title('Average Word Length Distribution in Reviews')
plt.xlabel('Word Length Intervals')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
# Annotate each bar with its exact percentage value
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height), xytext=(0, 3),
                 textcoords="offset points", ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
rounded_average_word_length_intervals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, float('inf')]
rounded_average_word_length_distribution = pd.cut(reviews['rounded_average_word_length'], bins=rounded_average_word_length_intervals).value_counts().sort_index()

plt.figure(figsize=(8, 6))
bars = plt.bar(rounded_average_word_length_distribution.index.astype(str), rounded_average_word_length_distribution.values)
plt.title('Rounded Average Word Length Distribution in Reviews')
plt.xlabel('Word Length Intervals')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
# Annotate each bar with its exact percentage value
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height), xytext=(0, 3),
                 textcoords="offset points", ha='center', va='bottom')

plt.tight_layout()
plt.show()

Sentence Count

In [ ]:
reviews.sentence_count.value_counts(normalize=True).sort_index().plot(title='Number of Sentences Distribution in Reviews', xlabel='Number of Sentences', ylabel='Frequency', xlim=(0,20))

Average Sentence Length

In [ ]:
reviews.average_sentence_length.value_counts(normalize=not True).sort_index().plot(title='Average Sentence Length Distribution in Reviews', xlabel='Average Sentence Length', ylabel='Frequency', xlim=(0,20))

Average Sentence Length

In [ ]:
reviews.rounded_average_sentence_length.value_counts(normalize=True).sort_index().plot(
    title='Rounded Average Sentence Length Distribution in Reviews',
    xlabel='Rounded Average Sentence Length',
    ylabel='Frequency',
    xlim=(0,50))

Lexical Diversity

In [ ]:
# Plot Lexical Diversity
plt.figure(figsize=(10, 6))
plt.hist(reviews['lexical_diversity'], bins=20, edgecolor='k')
plt.title('Lexical Diversity in Reviews')
plt.xlabel('Lexical Diversity')
plt.ylabel('Frequency')
plt.show()

Most Common Words

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
all_words = [word for sublist in reviews['most_common_words'].tolist() for word in sublist]

In [ ]:
fdist = FreqDist(all_words)

In [ ]:
common_words = fdist.most_common(10)

# Plot Most Common Words
plt.figure(figsize=(10, 6))
fdist.plot(30, cumulative=False)
plt.title('Word Frequency Distribution')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.show()

N-grams

In [ ]:
def get_ngrams(text, n):
    tokens = word_tokenize(text)
    n_grams = list(ngrams(tokens, n))
    return n_grams

n = 2  # Change 'n' to desired n-gram length
reviews['n_grams'] = reviews['text'].apply(lambda x: get_ngrams(str(x), n))
all_ngrams = [ngram for sublist in reviews['n_grams'].tolist() for ngram in sublist]
fdist_ngrams = FreqDist(all_ngrams)
common_ngrams = fdist_ngrams.most_common(10)